## Sharing machine learning models

### Save and retrieve Scikit-learn model

In [30]:
# Train a model.
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

iris = load_iris()
X, y = iris.data, iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y)
clr = RandomForestClassifier()
clr.fit(X_train, y_train)

# accuracy on test data with trained model
clr.score(X_test, y_test)

0.9473684210526315

In [31]:
# Convert into ONNX format
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType

initial_type = [('float_input', FloatTensorType([None, 4]))]
onx = convert_sklearn(clr, initial_types=initial_type)

# save trained model
with open("saved_models/rf_iris.onnx", "wb") as f:
    f.write(onx.SerializeToString())

In [32]:
# Compute the prediction with ONNX Runtime
import onnxruntime as rt
import numpy

# retrieve trained model
sess = rt.InferenceSession("saved_models/rf_iris.onnx")
input_name = sess.get_inputs()[0].name
label_name = sess.get_outputs()[0].name

# predict labels of test data
pred_onx = sess.run([label_name], {input_name: X_test.astype(numpy.float32)})[0]

In [33]:
pred_onx

array([0, 2, 1, 2, 1, 1, 1, 0, 1, 0, 0, 0, 2, 1, 2, 0, 1, 0, 0, 0, 1, 0,
       0, 2, 1, 0, 1, 2, 0, 2, 0, 1, 2, 2, 0, 0, 0, 0], dtype=int64)

In [34]:
from sklearn import metrics as score

# accuracy on test data using retrieved model
score.accuracy_score(y_test, pred_onx)

0.9473684210526315

### Save and retrieve Tensorflow model

In [46]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model
import tf2onnx

In [47]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Add a channels dimension
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")

In [48]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [49]:
class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = Conv2D(32, 3, activation='relu')
        self.flatten = Flatten()
        self.d1 = Dense(128, activation='relu')
        self.d2 = Dense(10)

    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

# Create an instance of the model
model = MyModel()

'dense_7'

In [50]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

In [51]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

In [52]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=True)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

In [53]:
EPOCHS = 2

for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
  train_loss.reset_states()
  train_accuracy.reset_states()
  print("Training epoch: {}".format(str(epoch+1)))
  for images, labels in train_ds:
    train_step(images, labels)

  print(
    f'Epoch {epoch + 1}, '
    f'Loss: {train_loss.result()}, '
    f'Accuracy: {train_accuracy.result() * 100}, '
  )

Training epoch: 1
Epoch 1, Loss: 0.13305729627609253, Accuracy: 95.99333190917969, 
Training epoch: 2
Epoch 2, Loss: 0.04282589256763458, Accuracy: 98.66166687011719, 


In [110]:
tf.saved_model.save(model, 'tf_model')

INFO:tensorflow:Assets written to: tf_model/assets


INFO:tensorflow:Assets written to: tf_model/assets


In [361]:
print(dir(model))

['_TF_MODULE_IGNORED_PROPERTIES', '__call__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_activity_regularizer', '_add_trackable', '_add_variable_with_custom_getter', '_assert_compile_was_called', '_assert_weights_created', '_auto_track_sub_layers', '_autocast', '_autographed_call', '_base_model_initialized', '_build_input_shape', '_call_accepts_kwargs', '_call_arg_was_passed', '_call_fn_arg_defaults', '_call_fn_arg_positions', '_call_fn_args', '_call_full_argspec', '_callable_losses', '_cast_single_input', '_check_call_args', '_checkpoint_dependencies', '_clear_losses', '_compile_was_called', '_compiled_trainable_state', '_compute_dtype', '_compute_dtype_ob

In [362]:
%%bash

python -m tf2onnx.convert --saved-model tf_model --output tfmodel.onnx

/opt/conda/lib/python3.9/runpy.py:127: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2021-06-15 08:38:16,024 - WARNING - '--tag' not specified for saved_model. Using --tag serve
2021-06-15 08:38:16,154 - INFO - Signatures found in model: [serving_default].
2021-06-15 08:38:16,155 - WARNING - '--signature_def' not specified, using first signature: serving_default
2021-06-15 08:38:16,155 - INFO - Output names: ['output_1']
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
2021-06-15 08:38:16,395 - WARNING - From /opt/conda/lib/python3.9/site-packages/tf2onnx/tf_loader.py:603: extract_sub_graph (from tensorflow.python.framework.graph_util_impl) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
2021-06-15 08:38:16,551 - INFO - Us

### Retrieve saved tensorflow model

In [473]:
import onnx
from onnx_tf.backend import prepare

In [474]:
loaded_model = onnx.load("tfmodel.onnx")

In [475]:
print(dir(loaded_model))

['ByteSize', 'Clear', 'ClearExtension', 'ClearField', 'CopyFrom', 'DESCRIPTOR', 'DiscardUnknownFields', 'Extensions', 'FindInitializationErrors', 'FromString', 'HasExtension', 'HasField', 'IsInitialized', 'ListFields', 'MergeFrom', 'MergeFromString', 'ParseFromString', 'RegisterExtension', 'SerializePartialToString', 'SerializeToString', 'SetInParent', 'UnknownFields', 'WhichOneof', '_CheckCalledFromGeneratedFile', '_SetListener', '__class__', '__deepcopy__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__unicode__', '_extensions_by_name', '_extensions_by_number', 'doc_string', 'domain', 'graph', 'ir_version', 'metadata_props', 'model_version', 'opset_import', 'producer_name', 'producer

In [476]:
tf_loaded_model = prepare(loaded_model)

In [477]:
print(dir(tf_loaded_model))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_graph', '_inputs', '_onnx_op_list', '_outputs', '_tensor_dict', '_tf_module', 'export_graph', 'graph', 'inputs', 'onnx_op_list', 'outputs', 'run', 'signatures', 'tensor_dict', 'tf_module']


In [478]:
tf_loaded_model.signatures

{'input_1:0': TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name='input_1_tf_0_1e47038a')}

In [479]:
#tf_loaded_model.export_graph("loaded_model")
#saved_loaded_model = tf.saved_model.load("loaded_model")
#print(dir(saved_loaded_model))

In [480]:
test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [481]:
tf_loaded_model.inputs

['input_1:0']

In [482]:
tf_loaded_model.outputs

['output_1']

In [483]:
tf_loaded_model.tensor_dict

{}

In [484]:
#%%bash

#onnx-tf convert -i tfmodel.onnx -o "loaded_model"

In [ ]:
import numpy as np

def test_step(images, labels):
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  # 'input_1:0': TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name='input_1_tf_0_da1f0fb4'
  # {"input_1:0": images[0], "dtype": tf.float32, "name": 'input_1_tf_0_da1f0fb4'}
  # output = prepare(onnx_model).run(input)
  # {'input_1:0': TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name='input_1_tf_0_58f50004')}
  #img_proto = tf.make_tensor_proto(images[0])
  #img_np = tf.make_ndarray(img_proto)
  #print(img_np)
  true_labels = []
  predicted_labels = []
  for i, item in enumerate(images):
      prediction = tf_loaded_model.run(images[i])
      #prediction = int(np.argmax(np.array(predictions).squeeze(), axis=0))
      true_labels.append(labels[i])
      predicted_labels.append(prediction)
  t_loss = loss_object(true_labels, predicted_labels)
  test_loss(t_loss)
  test_accuracy(true_labels, predicted_labels)

EPOCHS = 1

for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
  test_loss.reset_states()
  test_accuracy.reset_states()

  for test_images, test_labels in test_ds:
    ts = test_images #.numpy()
    test_step(ts, test_labels)

  print(
    f'Epoch {epoch + 1}, '
    f'Test Loss: {test_loss.result()}, '
    f'Test Accuracy: {test_accuracy.result() * 100}'
  )

In [ ]:
from tensorflow.python.platform import gfile
from tensorflow.core.protobuf import saved_model_pb2
from tensorflow.python.util import compat
from google.protobuf import text_format

graph_def = tf.compat.v1.GraphDef()

# These are set to the default names from exported models, update as needed.
filename = "loaded_model/saved_model.pb"

#INPUT_TENSOR_NAME = model.conv1.name + ":0"
#OUTPUT_TENSOR_NAME = model.d2.name + ":0"

print(INPUT_TENSOR_NAME, OUTPUT_TENSOR_NAME)

# Import the TF graph
with gfile.FastGFile(filename, 'rb') as f:
    data = compat.as_bytes(f.read())
    sm = saved_model_pb2.SavedModel()
    sm.ParseFromString(data)
    #graph_def = tf.GraphDef()
    #content = f.read()
    #a = text_format.Merge(content, graph_def)
    #graph_def.ParseFromString(content)
    with tf.Graph().as_default() as graph:
        #g_in = tf.import_graph_def(sm.meta_graphs[0].graph_def)
        #tf.import_graph_def(sm.meta_graphs[0].graph_def)
        tf.import_graph_def(sm.meta_graphs[0].graph_def, name="")
    
print(graph)
print(dir(graph))
#input_tensor = graph.get_tensor_by_name(INPUT_TENSOR_NAME)
#output_tensor = graph.get_tensor_by_name(OUTPUT_TENSOR_NAME)

In [171]:
INPUT_TENSOR_NAME = model.conv1.name
OUTPUT_TENSOR_NAME = model.d2.name

from tensorflow.python.platform import gfile

PB_PATH = "tf_model/saved_model.pb" #"loaded_model/saved_model.pb"

with gfile.FastGFile(PB_PATH,'rb') as f:
    graph_def = tf.compat.v1.GraphDef()
    graph_def.ParseFromString(f.read())

with tf.Graph().as_default() as graph:
    tf.import_graph_def(graph_def, name="")

input_tensor = graph.get_tensor_by_name(INPUT_TENSOR_NAME)
output_tensor = graph.get_tensor_by_name(OUTPUT_TENSOR_NAME)

DecodeError: Error parsing message

In [ ]:
from google.protobuf import text_format

INPUT_TENSOR_NAME = model.conv1.name
OUTPUT_TENSOR_NAME = model.d2.name

PB_PATH = "loaded_model/saved_model.pb"

with tf.io.gfile.GFile(PB_PATH,'r') as f:
  proto_b = f.read()
  graph_def = tf.compat.v1.GraphDef()
  text_format.Merge(proto_b, graph_def)
  graph_def.ParseFromString(graph_def)

with tf.Graph().as_default() as graph:
  tf.import_graph_def(graph_def, name="")

input_tensor = graph.get_tensor_by_name(INPUT_TENSOR_NAME)
output_tensor = graph.get_tensor_by_name(OUTPUT_TENSOR_NAME)